# Imagenet generation using **Stable Diffusion** v1.4 🎨 
*...using `🧨diffusers`*

Stable Diffusion is a text-to-image latent diffusion model created by the researchers and engineers from [CompVis](https://github.com/CompVis), [Stability AI](https://stability.ai/) and [LAION](https://laion.ai/). It's trained on 512x512 images from a subset of the [LAION-5B](https://laion.ai/blog/laion-5b/) database. This model uses a frozen CLIP ViT-L/14 text encoder to condition the model on text prompts. With its 860M UNet and 123M text encoder, the model is relatively lightweight and runs on a GPU with at least 10GB VRAM.

This Notebook shows my experiment with generation of Imagenet-like dataset using Stable Diffusion from Hugging Face [🧨 Diffusers library](https://github.com/huggingface/diffusers)

First, install `diffusers==0.2.4` as well `scipy`, `ftfy` and `transformers`

In [ ]:
!pip install diffusers==0.2.4
!pip install transformers scipy ftfy

Login into huggingface hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## Stable Diffusion pipeline

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", 
                                               torch_dtype=torch.float16, use_auth_token=True)  
pipe = pipe.to('cuda')

Get Imagenet classes

In [ ]:
with open('imagenet_idx_to_class.txt', 'r') as f:
    id_to_class = eval(f.read())

Output directory

In [ ]:
from pathlib import Path

out_dir = Path('./imagenet')
out_dir.mkdir(exist_ok=True, parents=True)

Generate images

In [ ]:
# count percentage of NSFW
n_total = 0
n_nsfw = 0

In [ ]:
import random
from torch import autocast

n_img_per_class = 5

for key, val in id_to_class.items():
    prompt = f'a photo of {val}, realistic, high quality, imagenet'
    
    for i in range(n_img_per_class):
        seed = random.randint(0, 10000)
        with autocast("cuda"):
            generator = torch.Generator("cuda").manual_seed(seed)    
            out = pipe(prompt, generator=generator)
            
        image = out["sample"][0]
        nsfw = out['nsfw_content_detected'][0]
            
        n_total += 1
        n_nsfw += int(nsfw)
        
        outpath = out_dir / f'{key}_{i}_seed_{seed}_nsfw_{nsfw}.jpg'
        image.save(outpath)

In [ ]:
print(f'Percentage of NSFW images: {n_nsfw / n_total * 100:.2f} %')